# 协程
- 参考资料
    - http://python.jobbole.com/86481/
    - http://python.jobbole.com/87310/
    - https://segmentfault.com/a/1190000009781688

# 迭代器
- 可迭代(lterable): 直接作用于for循环的变量
- 迭代器(lterator): 不但可以作用与for循环，还可以被next调用
- list是典型的可迭代对象,但不是迭代器
- 通过isinstance判断
- iterable和iterator可以转换
    - 通过iter函数

In [1]:
# 可迭代
l = [i for i in range(10)]

# l 是可迭代的,但不是迭代器
for idx in l:
    print(idx)
# range是个迭代器
for i in range(5):
    print(i)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


In [3]:
# isinstance案例
# 判断某个变量是否是一个实例

# 判断是否可迭代
from collections import Iterable
l1 = [1,2,3,4,5,6]

print(isinstance(l1, Iterable))

from collections import Iterator
print(isinstance(l1, Iterator))


True
False


In [6]:
# iter函数
s = "i love wangxiaojing"
print(isinstance(s, Iterable))
print(isinstance(s, Iterator))

s_iter = iter(s)
print(isinstance(s_iter, Iterable))
print(isinstance(s_iter, Iterator))
# Iterable:可迭代的
# Iterator:可迭代器

True
False
True
True


# 生成器
- generator: 一边循环一边计算下一个元素的机制/算法
- 需要满足三个条件:
    - 每次调用都生产出for循环需要的下一个元素
    - 如果达到最后一个后,爆出Stoplteration异常
    - 可以被next函数调用
- 如何生成一个生成器
    - 直接使用
    - 如果函数中包含yield,则这个函数就叫生成器
    - next调用函数,遇到yield返回

In [7]:
# 直接使用生成器
L = [x*x for x in range(5)] # 放在中括号中是列表生成器
g = (x*x for x in range(5)) # 放在小括号中就是生成器
print(type(L))
print(type(g))

<class 'list'>
<class 'generator'>


In [8]:
# 函数案例
def odd():
    print("Step 1")
    print("Step 2")
    print("Step 3")
    return None
odd()

Step 1
Step 2
Step 3


In [27]:
# 生成器案例
# 在函数odd中, yield负责返回
def odd():
    print("Step 1")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3
    
# odd() 是调用生成器
g = odd()
one = next(g)
print(one)

two = next(g)
print(two)

three = next(g)
print(three)

Step 1
1
Step 2
2
Step 3
3


In [12]:
# for 循环调用生成器
def fib(max):
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        print(b)
        a,b = b, a+b # 注意写法
        n += 1
    return "Done"
fib(5)

1
1
2
3
5


'Done'

In [13]:
# 斐波拉契数列的生成器写法
def fib(max):
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        yield b
        a, b = b, a+b # 注意写法
        n += 1
    # 需要注意,爆出异常是返回值是return的返回值
    return "Done"

g = fib(5)

for i in range(6):
    rst = next(g)
    print(rst)

1
1
2
3
5


StopIteration: Done

In [25]:
ge = fib(10)
'''
生辰器的典型用法是在for中使用
比较常用的典型生成器就是range
'''

for i in ge: # 在for循环中使用生成器
    print(i)
    
print(ge.__doc__)

1
1
2
3
5
8
13
21
34
55
None


# 协程
- 历史历程
    - 3.4引入协程,用yield实现
    - 3.5引入协程语法
    - 实现的协程比较好的包有asyncio,tornado,gevent
- 定义: 协程是为非抢占式多任务产生子程序的计算机程序组件,协程允许不同入口点在不同位置暂停或开始执行程序
- 从技术角度讲,协程就是一个你可以暂停执行的函数,或者干脆把协程理解成为生成器
- 协程的实现:
    - yield返回
    - send调用
- 协程的四个状态
    - inspect.getgeneratorstate(....)函数确定,该函数会返回下述字符串的一个:
    - GEN_CREATED:等待开始执行
    - GEN_RUNNING:解释器正在执行
    - GEN_SUSPENED:在yield表达式处暂停
    - GEN_CLOSED:执行结束
    - next预激(prime)
    - 代码案例v2
- 协程终止
    - 协程中未处理的异常会向上冒泡,传给next函数或send方法的调用方(即触发协程的对象)
    - 终止协程的一种方式:发送某个哨符值,让协程退出,内置的None和Ellipsis等常量经常用作哨符值===.
- yield from
    - 调用协程为了返回值,协程必须正常终止
    - 生成器正常终止会发出Stoplteration异常,异常对象的vlaue属性保存返回值
    - yield from 从内部捕获Stoplteration异常
    - 案例v03
    - 委派生成器
        - 包含yield from表达式的生成器函数
        - 委派生成器在yield form表达式处暂停,调用方可以直接把数据发给自生成器
        - 子生成器在把产生得到值发给调用方
        - 子生成器在最后,解释器会抛出Stoplteration,并且把返回值附加到异常对象上
        - 案例v04

In [1]:
# 协程代码案例1
def simple_coroutine():
    print('->start')
    x = yield
    print('->recibed', x)
# 主线程
sc = simple_coroutine()
print(11111)

# 可以使用sc.send(None),效果一样
next(sc) #预激

print(2222)
sc.send('zhexiao')

11111
->start
2222
->recibed zhexiao


StopIteration: 

In [2]:
# 案例02， 协程的状态
def simple_coroutine(a):
    print("-> start")
    
    b = yield a
    print('-> recived', a, b)
    
    c = yield a + b
    
    print('-> recived', a, b, c)
    
# runc 
sc = simple_coroutine(5)

aa = next(sc)
print(aa)
bb = sc.send(6) # 5, 6
print(bb)
cc = sc.send(7)# 5,6,7
print(cc)

-> start
5
-> recived 5 6
11
-> recived 5 6 7


StopIteration: 

In [3]:
# 案例 v03
def gen():
    for c in 'AB':
        yield c
# list直接用生成器作为参数
print(list(gen()))

def gen_new():
    yield from 'AB'
    
print(list(gen_new()))

['A', 'B']
['A', 'B']


In [15]:
# 案例v04,委派生成器
from collections import namedtuple
'''
解释:
1.外层for循环每次迭代会新建一个grouper实例,赋值给coroutine变量， grouper是委派生成器.
2.调用next(coroutine),预激委派生成器 grouper,此时进入while True循环,调用子生成器averanger后,在yield from表达式处暂停
3.内层for 循环调用 coroutine.send(value),直接把值传给子生成器averager.同时,当前的grouper实例(coroutine)在yield from表达式处暂停
4.内层循环结束后,grouper实例依旧在yield from表达式处暂停，因此,grouper函数定义体中为results[key]赋值得到语句还没有执行
5.coroutine.send(None)终止averager子生成器,子生成器抛出,StopIteration异常并将返回的数据包含在异常对象的value中,yield from可以直接抓取
StioItration异常并将对象的value赋值给results[key]
'''
ResClass = namedtuple('Res', 'count average')

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    
    while True:
        term = yield
        # None是哨兵值
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return ResClass(count, average)

# 委派生成器
def grouper(storages, key):
    while True:
        # 获取averager()返回的值
        storages[key] = yield from averager()
        
# 客户端代码
def client():
    process_data = {
        'boys_2': [1,3.5,5.6,8.6,3.2,5,9,3.2,4.2,4,54,4,8.65,2],
        'boys_1': [12.3, 45, 21, 1.45, 1.56, 3, 49, 5.67, 5.4, 6, 67, 21.3]
    }
    storages = []
    for k,v in process_data.items():
        # 获得协程
        coroutine = grouper(storages, k)
        # 预激协程
        next(coroutine)
        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)
        # 终止协程
        coroutine.send(None)
    print(storages)
# run
client()

TypeError: list indices must be integers or slices, not str

# asyncio
- python3.4开始引入标准库当中,内置对异步io的支持
- asyncio本身是一个消息循环
- 步骤
    - 创建消息循环
    - 把协程导入
    - 关闭
- 案例08
- 案例09-两个tasks
- 案例v10-得到多个网站

In [8]:
# 案例08
import  asyncio


@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    print("Start....")
    r = yield from asyncio.sleep(3)
    print("Done....")
    print("Hello again!")
def runEventLoop():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(hello())
    loop.close()

    
# 获取EventLoop
loop = asyncio.get_event_loop()
runEventLoop()
runEventLoop()
# 执行coroutine:
loop.run_until_complete(hello())
loop.close()

Hello world!
Start....
Done....
Hello again!
Hello world!
Start....
Done....
Hello again!


RuntimeError: Event loop is closed

In [2]:
import threading
import asyncio
@asyncio.coroutine
def hello():
    print("Hello world!(%s)" % threading.currentThread())
    print('Start...(%s)' % threading.currentThread())
    yield from asyncio.sleep(10)
    print('Done.....(%s)' % threading.currentThread())
    print('Hello again!(%s)' % threading.currrentThread())
    
loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: Event loop is closed

In [4]:
import asyncio

@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: Event loop is closed

# async and await
- 为了更好的表示异步
- python3.5开始引入
- 让coroutine代码更简洁
- 使用上,可以简单进行替换
    - 可以把 @asyncio.coroutine替换成async
    - yield from 替换成await
- 案例v11,把案例v09直接替换

In [5]:
import threading
import asyncio

#@asyncio.coroutine
async  def hello():
    print('Hello world! (%s)' % threading.currentThread())
    print('Start..... (%s)' % threading.currentThread())
    await asyncio.sleep(10)
    print('Done..... (%s)' % threading.currentThread())
    print('Hello again! (%s)' % threading.currentThread())

loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: Event loop is closed

# aiohttp
- 介绍
    - asyncio实现单线程并发IO,在客服端用处不大
    - 在服务器端可以asyncio+coroutine配合,因为http是io操作
    - asynico实现了TCP,UIDP,SSL等协议
    - aiohttp是给予asyncio实现的HTTP框架
    - pip install aiohttp
    - 案例07

In [ ]:
import asyncio

from aiohttp import web

async def index(request):
    await asyncio.sleep(0.5)
    return web.Response(body=b'<h1>Index</h1>')

async def hello(request):
    await asyncio.sleep(0.5)
    text = '<h1>hello, %s!</h1>' % request.match_info['name']
    return web.Response(body=text.encode('utf-8'))

async def init(loop):
    app = web.Application(loop=loop)
    app.router.add_route('GET', '/', index)
    app.router.add_route('GET', '/hello/{name}', hello)
    srv = await loop.create_server(app.make_handler(), '127.0.0.1', 8000)
    print('Server started at http://127.0.0.1:8000...')
    return srv

loop = asyncio.get_event_loop()
loop.run_until_complete(init(loop))
loop.run_forever()

/var/sw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: loop argument is deprecated
  from ipykernel import kernelapp as app
/var/sw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Application.make_handler(...) is deprecated, use AppRunner API instead


Server started at http://127.0.0.1:8000...


# concurrent.futures
- python3新增的库
- 类似其他语言的线程池的概念
- 此模块利用multiprocessiong实现真正的平行计算
- 核实原理:concurrent.futurer会以子进程的形式，平行的运行多个python解释器，从而令python程序可以利用多核CPU来提升速度.
    由于子进程与主解释器相分离,所以他们的全局解释器锁也是相互独立的,每个子进程都能够完整的使用一个CPU内核
- concurrent.futures.Executor
    - ThreadPoolExecutor
    - ProcessPoolExecutor
- submit(fn,args, kwargs)
    - fn:异步执行的函数
    - args.kwargs:参数
    - 案例 v14
    